In [1]:
import pandas as pd
import sklearn
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

## Import data

In [2]:
load_all = True
if load_all:
    # Load all
    df = pd.read_csv("all_episodes.csv", header=None)
else:
    # Load first 1000 lines
    df = pd.read_csv("all_episodes.csv", nrows=1000, header=None)
print(df.shape)

(59250, 147)


In [3]:
df.iloc[:, 143:148].describe()

,143,144,145,146
count,59250.000000,59250.000000,59250.0,59250.000000
mean,732.323797,0.113553,0.0,0.007994
std,130.638931,0.523060,0.0,0.026082
min,581.000000,0.000000,0.0,0.000000
25%,613.000000,0.000000,0.0,0.000000
50%,710.000000,0.000000,0.0,0.000000
75%,810.000000,0.000000,0.0,0.000000
max,1485.000000,9.000000,0.0,0.366667


In [4]:
features_nb = 113
actions_nb = 30
rewards_nb = 4
total_nb = features_nb + actions_nb + rewards_nb
print(features_nb, actions_nb, rewards_nb, '|', total_nb)

113 30 4 | 147


In [5]:
# Scale
scaler = StandardScaler()
scaler.fit(df)
df_scaled = pd.DataFrame(scaler.transform(df))

df_scaled.describe()

,0,1,2,3,4,5,6,7,8,9,...,137,138,139,140,141,142,143,144,145,146
count,5.925000e+04,5.925000e+04,5.925000e+04,5.925000e+04,5.925000e+04,59250.0,5.925000e+04,59250.0,5.925000e+04,5.925000e+04,...,59250.0,5.925000e+04,5.925000e+04,5.925000e+04,5.925000e+04,5.925000e+04,5.925000e+04,5.925000e+04,59250.0,5.925000e+04
mean,-5.295560e-14,-1.449710e-14,-3.840326e-15,-2.610551e-14,-1.297167e-14,0.0,6.790977e-14,0.0,-1.064776e-14,-2.859482e-14,...,0.0,-6.366497e-16,1.090420e-14,-8.773222e-15,-5.796105e-14,-4.078668e-15,2.946890e-14,5.994897e-14,0.0,2.850303e-14
std,1.000008e+00,1.000008e+00,1.000008e+00,1.000008e+00,1.000008e+00,0.0,1.000008e+00,0.0,1.000008e+00,1.000008e+00,...,0.0,1.000008e+00,1.000008e+00,1.000008e+00,1.000008e+00,1.000008e+00,1.000008e+00,1.000008e+00,0.0,1.000008e+00
min,-4.424635e-01,-4.824403e+00,-8.481953e-01,-1.338186e+01,-7.718636e-01,0.0,-3.315992e+00,0.0,-3.492009e+00,-2.241391e+00,...,0.0,-7.529335e-02,-4.884080e-02,-4.652972e-02,-4.523686e-02,-1.362676e-02,-1.158346e+00,-2.170951e-01,0.0,-3.064923e-01
25%,-4.424635e-01,1.184407e-01,-7.838490e-01,-3.682935e-01,-7.718636e-01,0.0,-5.518618e-01,0.0,-7.447552e-01,-8.479972e-01,...,0.0,-7.529335e-02,-4.884080e-02,-4.652972e-02,-4.523686e-02,-1.362676e-02,-9.133939e-01,-2.170951e-01,0.0,-3.064923e-01
50%,-4.424635e-01,1.184407e-01,-6.551564e-01,-3.682935e-01,-7.557383e-01,0.0,-5.518618e-01,0.0,-5.794156e-02,-8.479972e-01,...,0.0,-7.529335e-02,-4.884080e-02,-4.652972e-02,-4.523686e-02,-1.362676e-02,-1.708831e-01,-2.170951e-01,0.0,-3.064923e-01
75%,-4.424635e-01,2.682238e-01,1.275232e+00,9.509772e-02,1.278056e+00,0.0,-9.117352e-02,0.0,6.288720e-01,5.453964e-01,...,0.0,-7.529335e-02,-4.884080e-02,-4.652972e-02,-4.523686e-02,-1.362676e-02,5.945920e-01,-2.170951e-01,0.0,-3.064923e-01
max,8.719471e+00,2.215405e+00,3.919865e+00,5.192402e+00,1.668085e+00,0.0,7.740528e+00,0.0,6.810194e+00,4.609467e+00,...,0.0,1.328139e+01,2.047469e+01,2.149164e+01,2.210587e+01,7.338504e+01,5.761549e+00,1.698950e+01,0.0,1.375211e+01


In [6]:
# Split X, r
X = df_scaled.iloc[:, : features_nb+actions_nb]
r = df_scaled.iloc[:, features_nb+actions_nb : total_nb]

print(X.shape, r.shape)

# Offset
X = X.drop(index=len(X)-1, axis=0 # Drop the last one
r = r.drop(index=0, axis=0) # Drop the first one

print(X.shape, r.shape)

(59250, 143) (59250, 4)
(59249, 143) (59249, 4)


In [14]:
max(X.describe().loc['mean'])

2.267369423676867e-05

## Sklearn

In [7]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.multioutput import MultiOutputRegressor

In [8]:
X_train, X_test, r_train, r_test = train_test_split(X, r, test_size=0.2, random_state=42, shuffle=True)

In [9]:
X_train.shape, X_test.shape, r_train.shape, r_test.shape

((47399, 143), (11850, 143), (47399, 4), (11850, 4))

In [29]:
from sklearn.metrics import mean_squared_error, explained_variance_score, max_error, mean_absolute_error, mean_squared_log_error, r2_score, median_absolute_error

def metrics(model, X_test, y_test):
    y_pred = model.predict(X_test)
    print("\n-- METRICS --")
    
    metrics_list = [mean_squared_error, explained_variance_score, 
              mean_absolute_error, median_absolute_error, mean_squared_log_error, r2_score]
    
    for m in metrics_list:
        try:
            val = m(y_test, y_pred)
            print('{:30}{:5.4f}'.format(m.__name__, val))
        except:
            print('{:30}Cannot be computed'.format(m.__name__, val))

    

### RandomForestRegressor

In [31]:
max_depth = 30

regr_rf = RandomForestRegressor(n_estimators=100, max_depth=max_depth,
                                random_state=2)
regr_rf.fit(X_train, r_train)

metrics(regr_rf, X_test, r_test)


-- METRICS --
mean_squared_error            0.0084
explained_variance_score      0.9915
mean_absolute_error           0.0060
median_absolute_error         0.0001
mean_squared_log_error        Cannot be computed
r2_score                      0.9915


### Multioutput linear regression

In [32]:
regr_multilin = MultiOutputRegressor(LinearRegression())
regr_multilin.fit(X_train, r_train)

metrics(regr_multilin, X_test, r_test)


-- METRICS --
mean_squared_error            0.0129
explained_variance_score      0.9869
mean_absolute_error           0.0223
median_absolute_error         0.0069
mean_squared_log_error        Cannot be computed
r2_score                      0.9869


### Multioutput RFR

In [33]:
regr_multirfr = MultiOutputRegressor(RandomForestRegressor(n_estimators=100,
                                                          max_depth=max_depth,
                                                          random_state=0))

regr_multirfr.fit(X_train, r_train)

metrics(regr_multirfr, X_test, r_test)

KeyboardInterrupt: 

## Results analysis

In [39]:
r_test[144].describe()

count    11850.000000
mean        -0.003162
std          1.009807
min         -0.217095
25%         -0.217095
50%         -0.217095
75%         -0.217095
max         16.989500
Name: 144, dtype: float64

In [37]:
df_last_hits = X_test[r_test[144]!= 0]

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """Entry point for launching an IPython kernel.


IndexingError: Unalignable boolean Series provided as indexer (index of the boolean Series and of the indexed object do not match).

In [ ]:
df_last_hits.summary()